In [1]:
import os
import json
import ftplib
import datetime as dt

with open("credentials/passwords.json", "r") as file:
    passwords = json.load(file)

class InvalidPathError(Exception):
    pass

class Bluehost:
    def __init__(self):
        self.ftp = None

    def __enter__(self):
        self.ftp = self.login()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        try:
            if self.ftp:
                self.ftp.quit()
        finally:
            self.ftp = None
        return False

    def login(self):
        ftp_server = 'ftp.luh.sgg.mybluehost.me'
        ftp_username = 'mpuche2@luh.sgg.mybluehost.me'
        ftp_password = passwords["BLUEHOST"]
        self.ftp = ftplib.FTP(ftp_server)
        try:
            self.ftp.login(ftp_username, ftp_password)
            welcome_message = self.ftp.getwelcome()
            if welcome_message:
                print("===")
                print("FTP connection established successfully!")
                print("Welcome message:", welcome_message)
                print("===")
            else:
                print("FTP connection failed. Check your credentials.")
        except ftplib.all_errors as e:
            print("FTP connection failed:", e)
            self.ftp.quit()
        finally:
            return self.ftp

    def logout(self):
        if hasattr(self, 'ftp'):
            self.ftp.quit() 

    def file_exists(self, path):
        try:
            self.ftp.size(path)
            return True
        except ftplib.error_perm:
            return False

    def dir_exists(self, path):
        try:
            self.ftp.nlst(path)
            return True
        except ftplib.error_perm:
            return False

    def listdir_recursive_only_files(self, path = os.path.join("")):
        try:
            listdir = self.ftp.nlst(path)
        except Exception as e:
            print(f"Error listing dir {path}: {e}")
            return []

        cleaned_listdir = []
        for x in listdir:
            if x[0] == "." or x[-1] == ".":
                continue
            elif x.count('.') > 0:
                cleaned_listdir.append(x)
            else:
                cleaned_listdir.extend(self.listdir_recursive_only_files(x))

        cleaned_listdir = sorted(cleaned_listdir)
        return cleaned_listdir

    def listdir(self, path = os.path.join("")):
        try:
            listdir = self.ftp.nlst(path)
        except Exception as e:
            print(f"Error listing dir {path}: {e}")
            return []

        cleaned_listdir = []
        for x in listdir:
            if x == "." or x[-1] == "." or x == "cgi-bin":
                continue
            cleaned_listdir.append(x)

        cleaned_listdir = sorted(cleaned_listdir)
        return cleaned_listdir

    def upload_file(self, local_path, remote_path, overwrite=False):
        local_dir = os.path.dirname(local_path)
        remote_dir = os.path.dirname(remote_path)
        local_dir = local_dir if local_dir != "" else os.path.join(".")
        remote_dir = remote_dir if local_dir != "" else os.path.join(".")
        if not os.path.exists(local_dir):
            raise InvalidPathError(f"Local path dir doesn't exist: {local_dir}")
        if not os.path.exists(local_path):
            raise InvalidPathError(f"Local path doesn't exist: {local_path}")
        if not self.dir_exists(remote_dir):
            raise InvalidPathError(f"Remote dir doesn't exist: {remote_dir}")
        if not overwrite and self.file_exists(remote_path):
            raise InvalidPathError(f"Remote file already exists: {remote_path}")
        with open(local_path, 'rb') as file:
            response = self.ftp.storbinary(f"STOR {remote_path}", file)
        print("---")
        print(f"filename: {os.path.basename(local_path)}")
        print(f"local path: {local_path}")
        print(f"remote path: {remote_path}")
        print(f"response: {response}")

    def download_file(self, remote_path, local_path, overwrite=False):
        local_dir = os.path.dirname(local_path)
        remote_dir = os.path.dirname(remote_path)
        local_dir = local_dir if local_dir != "" else os.path.join(".")
        remote_dir = remote_dir if local_dir != "" else os.path.join(".")
        if not os.path.exists(local_dir):
            raise InvalidPathError(f"Local dir doesn't exist: {local_dir}")
        if not self.dir_exists(remote_dir):
            raise InvalidPathError(f"Remote dir doesn't exist: {remote_dir}")
        if not self.file_exists(remote_path):
            raise InvalidPathError(f"Remote file doesn't exists: {remote_path}")
        if os.path.exists(local_path) and overwrite == False:
            raise InvalidPathError(f"Local file already exist: {local_path}")
        if os.path.exists(local_path) and overwrite == "rename":
            os.rename(local_path, local_path + "_OLD_" + dt.datetime.now().strftime("_%Y%m%d%H%M%S"))
        with open(local_path, 'wb') as file:
            response = self.ftp.retrbinary('RETR ' + remote_path, file.write)
        print("---")
        print(f"filename: {os.path.basename(local_path)}")
        print(f"local path: {local_path}")
        print(f"remote path: {remote_path}")
        print(f"response: {response}")

In [2]:
def download_transcripts_from_bluehost():
    with Bluehost() as bh:
        # contents = bh.listdir(os.path.join(".", "transcriptions", "books", book))
        # print(contents)
        for book in ["B001", "B002", "B009"]:
            path = os.path.join(".", "transcriptions", "books", book, f"{book}_TRANS_ALL.txt")
            bh.download_file(
                remote_path = path,
                local_path = path,
                overwrite=True
            )

def update_website_in_bluehost():
    with Bluehost() as bh:
        for filename in ["index.html", "style.css", "script_slow.js", "script_fast.js"]:
            path = os.path.join(".", filename)
            bh.upload_file(
                remote_path = path,
                local_path = path,
                overwrite=True
            )

def update_website_dev_in_bluehost():
    with Bluehost() as bh:
        for filename in ["index_dev.html", "script_slow_dev.js"]:
            path = os.path.join(".", filename)
            bh.upload_file(
                remote_path = path,
                local_path = path,
                overwrite=True
            )

def create_folder():
    bh = Bluehost()
    ftp = bh.login()
    ftp.mkd('filters')
    ftp.close()

def upload_file():
    path = os.path.join(".", "filters", "filters.txt")
    with Bluehost() as bh:
        bh.upload_file(
            local_path = path,
            remote_path = path,
            overwrite="rename",
        )

def deleteme01():
    text = ""
    for book in ["B001", "B002", "B009"]:
        path = os.path.join(".", "transcriptions", "books", book, f"{book}_TRANS_ALL.txt")
        assert os.path.exists(path)
        with open(path, "r") as file:
            text += file.read()

    ans = ""
    lines = text.split("\n")
    for line in lines:
        if line.startswith("B0"):
            ans += "[o] " + line + "\n"

    with open("deleteme.txt", "w") as file:
        file.write(ans)

def deleteme02():
    with open("./filters/filters.txt", "r") as file:
        text = file.read(ans)

    lines = text.split("\n")
    for line in lines:
        if line.startswith("[0]"):
            ans += line + "\n"

    with open("./filters/included.txt", "w") as file:
        text = file.write(ans)

In [5]:
update_website_dev_in_bluehost()
update_website_in_bluehost()

===
FTP connection established successfully!
Welcome message: 220---------- Welcome to Pure-FTPd [privsep] [TLS] ----------
220-You are user number 6 of 150 allowed.
220-Local time is now 01:14. Server port: 21.
220-IPv6 connections are also welcome on this server.
220 You will be disconnected after 15 minutes of inactivity.
===
---
filename: index_dev.html
local path: ./index_dev.html
remote path: ./index_dev.html
response: 226-File successfully transferred
226 0.129 seconds (measured here), 19.77 Kbytes per second
---
filename: script_slow_dev.js
local path: ./script_slow_dev.js
remote path: ./script_slow_dev.js
response: 226-File successfully transferred
226 0.256 seconds (measured here), 123.31 Kbytes per second
===
FTP connection established successfully!
Welcome message: 220---------- Welcome to Pure-FTPd [privsep] [TLS] ----------
220-You are user number 6 of 150 allowed.
220-Local time is now 01:14. Server port: 21.
220-IPv6 connections are also welcome on this server.
220 You 

In [4]:
# update_website_in_bluehost()

!git add .
!git commit -m message
!git push
!git status

!git add .
!git commit -m message
!git push
!git status

[main 986dde2] message
 2 files changed, 29 insertions(+), 37 deletions(-)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 696 bytes | 696.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/mpuche3/englishipa
   7f2b79d..986dde2  main -> main
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [ ]:
# with Bluehost() as bh:
#     # bh.ftp.rename('./audio/books/B014', './audio/books/DELETEME_B014')
#     bh.ftp.mkd('./audio/books/B014')
#     path_text = os.path.join(".", "text", "books")
#     path_tran = os.path.join(".", "transcriptions", "books")
#     path_audio = os.path.join(".", "audio", "books")
#     contents = bh.ftp.nlst(path_audio)
#     print(contents)

In [ ]:
# for i in range(14, 15):
#     book = "B0" + str(i).zfill(2)
#     # os.mkdir(os.path.join(".", "text", "books", book))
#     # os.mkdir(os.path.join(".", "transcriptions", "books", book))
#     with Bluehost() as bh:
#         bh.download_file(
#             remote_path= os.path.join(".", "text", "books", book, f'{book}_TEXTS_ALL.txt'),
#             local_path = os.path.join(".", "text", "books", book, f'{book}_TEXTS_ALL.txt'),
#             overwrite = "rename"
#         )
#         bh.download_file(
#             remote_path= os.path.join(".", "transcriptions", "books", book, f'{book}_TRANS_ALL.txt'),
#             local_path = os.path.join(".", "transcriptions", "books", book, f'{book}_TRANS_ALL.txt'),
#             overwrite = "rename"
#         )

In [ ]:
# book = "B014"
# os.mkdir(os.path.join(".", "text", "books", book))
# os.mkdir(os.path.join(".", "transcriptions", "books", book))
# with Bluehost() as bh:
#     bh.download_file(
#         remote_path= os.path.join(".", "text", "books", book, f'{book}_TEXTS_ALL.txt'),
#         local_path = os.path.join(".", "text", "books", book, f'{book}_TEXTS_ALL.txt'),
#         overwrite = "rename"
#     )
#     bh.download_file(
#         remote_path= os.path.join(".", "transcriptions", "books", book, f'{book}_TRANS_ALL.txt'),
#         local_path = os.path.join(".", "transcriptions", "books", book, f'{book}_TRANS_ALL.txt'),
#         overwrite = "rename"
#     )

In [ ]:
# path = os.path.join(".", "text", "books", "B004")
# with Bluehost() as bh:
#     contents = bh.listdir(path)
# print(contents)

In [ ]:
# path = os.path.join(".", "filters", "filters.txt")
# with Bluehost() as bh:
#     bh.upload_file(
#         remote_path = path,
#         local_path = path,
#         # overwrite=True
#     )

In [ ]:
# bh = Bluehost()
# ftp = bh.login()

In [ ]:
# path = os.path.join(".", "text", "books")
# bh.listdir(path)

In [ ]:
# book = "B009"
# path = os.path.join(".", "text", "books", book, f"{book}_TEXTS_ALL.txt")
# bh.upload_file(
#     local_path = path,
#     remote_path = path
# )

In [ ]:
# bh.logout()

In [ ]:
# upload_file()

In [ ]:
def generate_B001():
    path = os.path.join(".", "filters", "filters.txt")
    with open(path, "r") as file:
        filters = file.read()

    filters = filters.strip().split("\n")
    filters = [filter for filter in filters if "[o] B001" in filter]
    filters = [filter[4:4+8] for filter in filters]

    path = os.path.join(".", "transcriptions", "books", "B001", "B001_TRANS_ALL.txt")
    with open(path, "r") as file:
        lines = file.read().strip().split("\n")

    book = ""
    include = True
    for line in lines:
        if line.startswith("B001"):
            if line[0: 8] in filters:
                include = True
            else:
                include =False
        if include == True:
            book += line + "\n"

    with open(os.path.join(".", "B001.txt"), "w") as file:
        file.write(book)